# Installation and Imports

Install and import everything we need.

In [ ]:
!pip install ydata-profiling

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from ydata_profiling import ProfileReport

# Minimal Working Example - Profiling

Check whether the library is working.

In [ ]:
df = pd.DataFrame(np.random.rand(100, 5), columns=["a", "b", "c", "d", "e"])
profile = ProfileReport(df, title="Report", explorative=True)
profile.to_notebook_iframe()
profile.to_file("report.html")


# Dataset Import

Examples of various dataset imports.

## Sklearn Datasets

In [ ]:
from sklearn.datasets import fetch_california_housing
data = fetch_california_housing(as_frame=True)
df = data.frame
df.head()

## Seaborn Dataset

In [ ]:
import seaborn as sns
sns.get_dataset_names()


In [ ]:
df = sns.load_dataset("flights")
df.head()

## Statsmodels Datasets

In [ ]:
import statsmodels.api as sm
df = sm.datasets.get_rdataset("AirPassengers").data
df.head()

# Dataset Example Profiling - Titanic


In [ ]:
import seaborn as sns
df = sns.load_dataset("titanic")
ProfileReport(df, title="Report", explorative=True).to_notebook_iframe()

# Dataset Example Profiling - Adult Income

In [ ]:
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data"
df = pd.read_csv(url, header=None, na_values="?", skipinitialspace=True)
ProfileReport(df, title="Report", explorative=True).to_notebook_iframe()

# Data Quality Improvement Example - Dummy Customer Data



## Dummy Dataset Generartion

In [ ]:
import pandas as pd
import numpy as np

data = {
    "ID": range(1, 21),
    "Age": [25, 102, np.nan, 40, 28, 15, 300, 22, 33, np.nan, 27, 35, 41, 22, 22, 25, 26, 25, 120, 17],
    "Gender": ["M", "M", "F", None, "M", "Male", "femlae", "M", "F", "F", "M", "M", "F", "F", "M", None, "M", "M", "F", "F"],
    "Income": [45000, -10000, 21050, 67000, 43100, 16000, 110000, np.nan, 27000, 26000, 28000, 29000, None, 22000, 31000, 1001000, 32500, 33000, 34800, 35000],
    "Purchases": [5, 3, np.nan, 0, 100, 2, 1, 3, 2, 3, 1, 3, 3, 5, 3, 4, 3, 5, 3, 3],
    "Country": ["CZ", "DE", "FR", "UK", None, "CZ", "CZ", "UK", "CZ", "FR", "CZ", "CZ", "DE", "CZ", "CZ", "CZ", "CZ", "CZ", "CZ", "CZ"],
    "Rating": [4, 5, 6, 3, np.nan, 2, 1, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4],
}

df = pd.DataFrame(data)
df.to_csv("uncleaned_dataset.csv", index=False)
df.head(21)

## Data Profilling

In [ ]:
profile = ProfileReport(df, title="Report", explorative=True)
profile.to_notebook_iframe()

## Fixing the Issues

In [ ]:
df.set_index("ID", drop=True, inplace=True)
df.head(21)

In [ ]:
min_age, max_age = 18, 110
df.loc[(df["Age"] < min_age) | (df["Age"] > max_age), "Age"] = np.nan
df.head(21)

In [ ]:
max_purchases = 10
df.loc[(df["Purchases"] > max_purchases), "Purchases"] = np.nan
df.head(21)

In [ ]:
min_income, max_income = 0, 500000
df.loc[(df["Income"] < min_income) | (df["Income"] > max_income), "Income"] = np.nan
df.head(21)

In [ ]:
df["Rating"] = df["Rating"].fillna(df["Rating"].median())
df.head(21)

In [ ]:
clean_df = df.dropna()
clean_df

In [ ]:
profile = ProfileReport(clean_df, title="Report", explorative=True)
profile.to_notebook_iframe()

# Data Quality Improvement Example - Time Series

## Data Creation and Profilling

In [ ]:
def generate_time_series_dataset(n=200, freq="1h", seed=42):
    np.random.seed(seed)

    # --- Time index ---
    time_index = pd.date_range("2025-01-01", periods=n, freq=freq)

    # --- Base signal (trend + noise) ---
    base = np.linspace(0, 10, n) + np.random.normal(0, 0.5, n)

    # --- Derived signals ---
    signal_A = np.sin(base) + np.random.normal(3, 0.1, n)
    signal_B = np.cos(base) + np.random.normal(-2, 0.5, n)
    signal_C = 0.5 * signal_A + 0.3 * np.random.normal(1, 3, n)  # correlated with A
    signal_D = np.random.normal(-100, 10, n)                         # uncorrelated noise

    df = pd.DataFrame({
        "time": time_index,
        "signal_A": signal_A,
        "signal_B": signal_B,
        "signal_C": signal_C,
        "signal_D": signal_D,
    }).set_index("time")

    # --- Randomly drop some rows (simulate missing timestamps) ---
    drop_idx = np.random.choice(df.index, size=int(0.1 * n), replace=False)
    df = df.drop(drop_idx)

    # --- Inject NaNs into random cells ---
    for col in df.columns:
        nan_idx = np.random.choice(df.index, size=int(0.05 * len(df)), replace=False)
        df.loc[nan_idx, col] = np.nan

    # --- Add duplicate rows ---
    dup_idx = np.random.choice(df.index, size=int(0.05 * len(df)), replace=False)
    duplicates = df.loc[dup_idx]
    df = pd.concat([df, duplicates]).sort_index()

    return df

df = generate_time_series_dataset()
profile = ProfileReport(df, title="Report", explorative=True)
profile.to_notebook_iframe()

## Fixing the Issues

### Duplicate Rows - Removal

In [ ]:
df = df[~df.index.duplicated(keep="first")]

### Missing Rows - Reindexing

In [ ]:
full_idx = pd.date_range(df.index.min(), df.index.max(), freq="1h")
df = df.reindex(full_idx)   # missing timestamps will be NaN
df.index.name = "time"

### Missing Cells - Interpolation

In [ ]:
df = df.interpolate(method="time", limit_direction="both")

### Normalization (If Required)

In [ ]:
df = (df - df.mean()) / df.std()

### Normalization and Scaling (More Examples)

In [ ]:
from sklearn.preprocessing import StandardScaler

np.random.seed(42)
x = np.linspace(0, 10, 100)
y = 50 + 20 * np.sin(x) + np.random.normal(0, 5, size=len(x))  # offset + large std

df = pd.DataFrame({"x": x, "y": y})

scaler = StandardScaler()
df["y_zscore"] = scaler.fit_transform(df[["y"]])

fig, axes = plt.subplots(2, 2, figsize=(12, 8))

# Line plots
axes[0, 0].plot(df["x"], df["y"], color="orange")
axes[0, 0].set_title("Original Data (Offset + High Std)")
axes[0, 0].set_xlabel("x")
axes[0, 0].set_ylabel("y")
axes[0, 0].grid(True)

axes[0, 1].plot(df["x"], df["y_zscore"], color="blue")
axes[0, 1].set_title("After Z-Score Normalization")
axes[0, 1].set_xlabel("x")
axes[0, 1].set_ylabel("z-score")
axes[0, 1].grid(True)

# Histograms
axes[1, 0].hist(df["y"], bins=20, color="orange", edgecolor="black")
axes[1, 0].set_title("Original Data Distribution")
axes[1, 0].set_xlabel("Value")
axes[1, 0].set_ylabel("Frequency")
axes[1, 0].grid(True, alpha=0.3)

axes[1, 1].hist(df["y_zscore"], bins=20, color="blue", edgecolor="black")
axes[1, 1].set_title("Z-Scored Data Distribution")
axes[1, 1].set_xlabel("Z-Score")
axes[1, 1].set_ylabel("Frequency")
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()


In [ ]:
from sklearn.preprocessing import MinMaxScaler

np.random.seed(42)
x = np.linspace(0, 10, 100)
y = np.random.uniform(0, 255, size=len(x))

df = pd.DataFrame({"x": x, "y": y})

scaler = MinMaxScaler()
df["y_scaled"] = scaler.fit_transform(df[["y"]])

fig, axes = plt.subplots(2, 2, figsize=(12, 8))

axes[0, 0].plot(df["x"], df["y"], color="orange")
axes[0, 0].set_title("Original Data (Uniform 0–255)")
axes[0, 0].set_xlabel("x")
axes[0, 0].set_ylabel("Value")
axes[0, 0].grid(True)

axes[0, 1].plot(df["x"], df["y_scaled"], color="blue")
axes[0, 1].set_title("After Min–Max Scaling (0–1)")
axes[0, 1].set_xlabel("x")
axes[0, 1].set_ylabel("Scaled Value")
axes[0, 1].grid(True)

axes[1, 0].hist(df["y"], bins=20, color="orange", edgecolor="black")
axes[1, 0].set_title("Original Data Distribution")
axes[1, 0].set_xlabel("Value")
axes[1, 0].set_ylabel("Frequency")
axes[1, 0].grid(True, alpha=0.3)

axes[1, 1].hist(df["y_scaled"], bins=20, color="blue", edgecolor="black")
axes[1, 1].set_title("Min–Max Scaled Distribution")
axes[1, 1].set_xlabel("Scaled Value")
axes[1, 1].set_ylabel("Frequency")
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Encoding

In [ ]:
from sklearn.preprocessing import OrdinalEncoder

# --- Original dataset ---
df = pd.DataFrame({
    "gene": [
        "GAA_01", "GAA_02", "GAB_01", "GAB_02", "GAC_05",
        "GAA_03", "GAC_01", "GAB_03", "GAD_01", "GAA_10"
    ],
    "color": ["red", "green", "blue", "red", "green", "blue", "red", "green", "blue", "red"],
    "satisfaction": ["low", "medium", "high", "medium", "low", "high", "medium", "high", "low", "medium"],
    "country": ["CZ", "DE", "CZ", "DE", "CZ", "DE", "CZ", "DE", "CZ", "DE"],
    "product_type": ["A", "B", "A", "C", "B", "A", "C", "B", "C", "A"],
    "sales": [200, 310, 250, 420, 300, 215, 480, 330, 410, 260]
})

print("🧬 Original DataFrame:\n")
print(df, "\n")

# --- Semi-ordinal encoding for gene ---
families = sorted({g.split("_")[0][1:] for g in df["gene"]})  # e.g. ["AA", "AB", "AC", "AD"]
family_to_index = {fam: i for i, fam in enumerate(families)}

def gene_semi_ordinal_score(gene_code: str) -> int:
    family, variant = gene_code.split("_")
    fam_key = family[1:] if family.startswith("G") else family
    fam_index = family_to_index.get(fam_key, -1)
    return fam_index * 100 + int(variant)

df["gene_enc"] = df["gene"].map(gene_semi_ordinal_score)

# --- Binary encoding for country ---
df["country_enc"] = df["country"].map({"CZ": 0, "DE": 1})

# --- One-hot encoding for color ---
df_color = pd.get_dummies(df["color"], prefix="color")

# --- Ordinal encoding for satisfaction ---
ord_enc = OrdinalEncoder(categories=[["low", "medium", "high"]])
df["satisfaction_enc"] = ord_enc.fit_transform(df[["satisfaction"]]).astype(int)

# --- Target / mean encoding for product_type ---
product_mean = df.groupby("product_type")["sales"].mean().to_dict()
df["product_type_enc"] = df["product_type"].map(product_mean)

# --- Combine encoded dataset ---
df_encoded = pd.concat([
    df[["gene_enc", "country_enc", "satisfaction_enc", "product_type_enc", "sales"]],
    df_color
], axis=1)

print("🔢 Encoded DataFrame:\n")
print(df_encoded)

## Final Profilling

In [ ]:
profile = ProfileReport(df, title="Report", explorative=True)
profile.to_notebook_iframe()

# Feature Extraction / Dimmension Reduction

Examples with PCA and LDA

## Principal Component Analysis

In [ ]:
# Time index
t = np.linspace(0, 10, 500)  # 500 samples from 0 to 10 seconds

# Create 5 unique base signals (sinusoids with different frequencies and phases)
base_signals = {
    f"signal_{i+1}": np.sin(t * (i+1) + i)
    for i in range(5)
}

# Create correlated variants (each base signal gets 4 noisy copies)
data = {}
for name, base in base_signals.items():
    for j in range(4):
        data[f"{name}_var{j+1}"] = base + np.random.normal(0, 0.05, len(base))  # small Gaussian noise

# Combine into DataFrame
df = pd.DataFrame(data, index=pd.Index(t, name="time"))

# Add some artificial imperfections
df = df.sort_index()  # and sort back by time

profile = ProfileReport(df, title="Report", explorative=True)
profile.to_notebook_iframe()


In [ ]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

scaler = StandardScaler()
X_scaled = scaler.fit_transform(df)

n = 5
pca = PCA(n_components=n)
X_pca = pca.fit_transform(X_scaled)
df_pca = pd.DataFrame(X_pca, index=df.index, columns=[f"PC{i+1}" for i in range(n)])

pca_full = PCA(n_components=df.shape[1])
pca_full.fit(X_scaled)

explained = pca_full.explained_variance_ratio_
cumulative = np.cumsum(explained)

fig, axes = plt.subplots(3, 1, figsize=(8, 9), sharex=False)

# Original signals
axes[0].plot(df.index, df, alpha=0.5)
axes[0].set_title("Original 20 Correlated Signals")
axes[0].set_ylabel("Amplitude")

# Top n PCA components
axes[1].plot(df_pca.index, df_pca)
axes[1].set_title(f"Top {n} Principal Components (PCA)")
axes[1].set_xlabel("Time")
axes[1].set_ylabel("Component Value")

# Explained variance
axes[2].bar(range(1, len(explained)+1), explained, alpha=0.7, label="Individual variance")
axes[2].plot(range(1, len(cumulative)+1), cumulative, color='r', marker='o', label="Cumulative variance")
axes[2].set_title("Explained Variance by Principal Components")
axes[2].set_xlabel("Principal Component")
axes[2].set_ylabel("Explained Variance Ratio")
axes[2].legend()

plt.tight_layout()
plt.show()

## Linear Discriminant Analysis

In [ ]:
# --- Synthetic signal dataset ---
np.random.seed(0)
n_samples = 100
t = np.linspace(0, 2*np.pi, 10)  # 10 time points per sample

def make_class(base_amp, noise, phase_shift):
    """Generate noisy sinusoidal samples for one class."""
    return base_amp * np.sin(t + phase_shift) + np.random.normal(0, noise, (n_samples, len(t)))

# More overlapping amplitudes and random phase shifts
normal = make_class(1.0, 0.15, 0.0)
overheated = make_class(1.2, 0.15, 0.2)
underloaded = make_class(0.8, 0.15, -0.3)

# Combine
X = np.vstack([normal, overheated, underloaded])
y = np.array(["Normal"]*n_samples + ["Overheated"]*n_samples + ["Underloaded"]*n_samples)

# Put into DataFrame
df = pd.DataFrame(X, columns=[f"t{i+1}" for i in range(X.shape[1])])
df["state"] = y

profile = ProfileReport(df, title="Report", explorative=True)
profile.to_notebook_iframe()



In [ ]:
import matplotlib.pyplot as plt
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA

# Separate features and target
X = df.drop(columns="state").values
y = df["state"].values

# Fit LDA (reduce to 2D for visualization)
lda = LDA(n_components=2)
X_lda = lda.fit_transform(X, y)

# Plot LDA projection
plt.figure(figsize=(8, 5))
for label, color in zip(["Normal", "Overheated", "Underloaded"], ["blue", "red", "green"]):
    plt.scatter(X_lda[y == label, 0], X_lda[y == label, 1], label=label, alpha=0.7)
plt.xlabel("LDA Component 1")
plt.ylabel("LDA Component 2")
plt.title("Linear Discriminant Analysis of Machine States")
plt.legend()
plt.grid(True)
plt.show()

# Data Stratification

## Simple Stratification

In [ ]:
from sklearn.model_selection import train_test_split

# Example dataset
df = pd.DataFrame({
    "feature1": [1, 2, 3, 4, 5, 6, 7, 8],
    "feature2": [10, 20, 30, 40, 50, 60, 70, 80],
    "label":    ["A", "A", "B", "B", "A", "A", "B", "B"]
})

# Split DataFrame while keeping class ratios the same
train_df, test_df = train_test_split(
    df,
    test_size=0.25,
    stratify=df["label"],   # stratify by label column
    random_state=42
)

print("Training")
print(train_df.head())
print("Validation")
print(test_df.head())

## Binning Stratification

In [ ]:
from sklearn.model_selection import train_test_split

df = pd.DataFrame({
    "age": [12, 18, 25, 33, 37, 42, 47, 52, 56, 61, 65, 70, 75, 19, 22, 28, 39, 44, 58, 63, 18],
    "income": [25000, 27000, 32000, 35000, 37000, 40000, 42000, 45000, 48000, 39000,
               50000, 52000, 54000, 55000, 29000, 31000, 33000, 38000, 41000, 47000, 51000]
})

df["age_bin"] = pd.cut(df["age"], bins=[0, 20, 40, 60, 80], labels=["<20", "20–40", "40–60", "60–80"])

train, test = train_test_split(
    df,
    test_size=0.3,
    stratify=df["age_bin"],
    random_state=42
)

print("Training set:")
print(train[["age", "age_bin"]].sort_values("age"))

print("\nTest set:")
print(test[["age", "age_bin"]].sort_values("age"))

print("\nAge bin distribution:")
print("Train:")
print(train["age_bin"].value_counts(normalize=True))
print("Test:")
print(test["age_bin"].value_counts(normalize=True))


## Compound Stratification

In [ ]:
from sklearn.model_selection import train_test_split

df = pd.DataFrame({
    "age": np.random.randint(18, 75, size=500),
    "income": np.random.normal(40000, 12000, size=500).astype(int),
    "employment": np.random.choice(["employed", "unemployed"], size=500, p=[0.8, 0.2])
})

df["age_group"] = pd.cut(
    df["age"], bins=[0, 20, 40, 60, 80], labels=["<20", "20–40", "40–60", "60–80"]
)

df["strata"] = df["employment"].astype(str) + "_" + df["age_group"].astype(str)

train, test = train_test_split(
    df,
    test_size=0.3,
    stratify=df["strata"],
    random_state=42
)

print("Training set:")
print(train[["age", "age_group", "employment", "strata"]].sort_values("age"))

print("\nTest set:")
print(test[["age", "age_group", "employment", "strata"]].sort_values("age"))

print("\nStrata distribution (train vs test):")
print(train["strata"].value_counts(normalize=True).sort_index())
print(test["strata"].value_counts(normalize=True).sort_index())


# Automated Pipelines

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# --- 1️⃣ Generate random data with missing values ---
np.random.seed(42)
n = 30

df = pd.DataFrame({
    "age": np.random.randint(18, 60, size=n).astype(float),
    "income": np.random.normal(50000, 8000, size=n),
    "gender": np.random.choice(["M", "F", None], size=n, p=[0.45, 0.45, 0.10]),
    "city": np.random.choice(["Prague", "Berlin", "Paris", None], size=n, p=[0.3, 0.3, 0.3, 0.1])
})

# Introduce additional missing values
df.loc[np.random.choice(df.index, 5, replace=False), "age"] = np.nan
df.loc[np.random.choice(df.index, 5, replace=False), "income"] = np.nan

print("🧾 Original DataFrame (first 10 rows):\n")
print(df.head(10), "\n")

# --- 2️⃣ Split into train and test sets manually ---
df_train = df.iloc[:20].copy()
df_test = df.iloc[20:].copy()

# --- 3️⃣ Define preprocessing pipeline ---
numeric_features = ["age", "income"]
categorical_features = ["gender", "city"]

numeric_transformer = Pipeline([
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler())
])

categorical_transformer = Pipeline([
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("encoder", OneHotEncoder(handle_unknown="ignore"))
])

preprocessor = ColumnTransformer([
    ("num", numeric_transformer, numeric_features),
    ("cat", categorical_transformer, categorical_features)
])

# --- 4️⃣ Fit the pipeline on train data and transform both ---
X_train_processed = preprocessor.fit_transform(df_train)
X_test_processed = preprocessor.transform(df_test)

# --- 5️⃣ Convert results back to DataFrames for clarity ---
train_cols = (
    numeric_features +
    list(preprocessor.named_transformers_["cat"].named_steps["encoder"].get_feature_names_out(categorical_features))
)
df_train_processed = pd.DataFrame(X_train_processed, columns=train_cols)
df_test_processed = pd.DataFrame(X_test_processed, columns=train_cols)

print("✅ Processed Training Data (first 5 rows):\n")
print(df_train_processed.head(), "\n")

print("🧩 Processed Test Data (first 5 rows):\n")
print(df_test_processed.head())

